In [1]:
### gets distribution of committees between three blocks
### goal: minimize pairwise interactions between members
### to reduce spread of COVID-19

class graph():
    def __init__(self, dataframe):
        self.matrix = dataframe.set_index("MC")
        self.transpose = transpose = self.matrix.transpose()
        
        self.members = set( self.matrix.columns )
        self.committees = set( self.transpose.columns )

    def member_adjacency(self, member):
        neighbours = set( self.matrix[ self.matrix[member] == 1 ].index )
        neighbours = neighbours.intersection( self.committees )
        
        return neighbours
    
    def committee_adjacency(self, committee):
        neighbours = set( self.transpose[ self.transpose[committee] == 1 ].index )
        neighbours = neighbours.intersection( self.members )
        
        self.members = self.members.difference( neighbours )
        return neighbours, len( neighbours )
    
    def connected_(self, queue):
        component, weight = [], 0  
        
        while queue:
            committee = queue.pop()
            component.append( committee )
            self.committees.remove( committee )

            members, w = self.committee_adjacency( committee )
            weight = weight + w
            
            queue = queue.union( *[ self.member_adjacency( member ) for member in members ] )
            queue = self.committees.intersection( queue )

        return component, weight
    
    def components(self):
        components = []
        while self.committees:
            component = list( self.committees )[0]
            components.append( self.connected_( set([component]) ) )
        return components

In [2]:
def paritition( dataset ):
    components = graph( dataset ).components()
    
    if len(components) < 3:
        return "FAIL"
    
    components.sort( key = lambda x: x[1], reverse = True )
    slates = [ ([], 0) for x in range(3) ]
    
    while components:
        minimum = slates.index( min(slates, key = lambda x: x[1]) )
        committees, weight = components.pop(0)
        
        current, sum_weight = slates[ minimum ]
        slates[ minimum ] = ( current + committees, sum_weight + weight )
        
    return slates

In [3]:
class partitions:
    def __init__(self, dataset, N = 3):
        dataset = dataset.set_index("MC").transpose()
        self.committees = [ (x, set(dataset[ dataset[x] == 1 ].index)) for x in dataset.columns ]
        self.partitions = [ [[], set()] for x in range(N) ]

    def weight(self, current, index):
        return len( current[1].difference( self.partitions[index][1] ) )

    def add(self, current, index):
        self.partitions[index][0].append( current[0] )
        self.partitions[index][1] = self.partitions[index][1].union( current[1] )
        self.committees.remove( current )
        
    def partition(self):
        while self.committees:
            index   = self.partitions.index( min(self.partitions, key = lambda x: len(x[1])) )
            current = min( self.committees, key = lambda x: self.weight(x, index) )
            
            self.add( current, index )
            
        return [ (x[0], len( x[1] )) for x in self.partitions ]

In [13]:
def run( dataset, N = 3 ):
    return partitions( dataset, N ).partition()

run(dataset)

[(['HA00', 'RU00', 'SM00', 'ED00', 'BU00', 'GO00', 'PW00', 'BA00'], 222),
 (['SO00', 'IE00', 'VR00', 'WM00', 'SY00', 'AG00', 'AS00', 'IF00'], 237),
 (['ZI00', 'IG00', 'HM00', 'JU00', 'FA00', 'II00', 'AP00'], 201)]

In [8]:
import pandas as pd 
dataset = pd.read_csv("D:\\OneDrive - Bipartisan Policy Center\\Congress\\COVID-19\\ComxMC116.csv")

In [9]:
print("Optimal Approximation: \n\n")
for N in range(1, 4):
    trial = run( dataset, N )
    for slate in range(N):
        print("Slate {}: {}".format(slate + 1, trial[slate][0]) )
    print( "\nTotal pairwise interactions with {} slates: {} \n".format( N, sum([ x[1] ** 2 for x in trial ]) ) )

Optimal Approximation: 


Slate 1: ['HA00', 'SO00', 'ZI00', 'RU00', 'IE00', 'IG00', 'SM00', 'VR00', 'HM00', 'JU00', 'FA00', 'SY00', 'ED00', 'GO00', 'AG00', 'PW00', 'AS00', 'II00', 'BU00', 'WM00', 'BA00', 'IF00', 'AP00']

Total pairwise interactions with 1 slates: 187489 

Slate 1: ['HA00', 'ZI00', 'IE00', 'IG00', 'HM00', 'JU00', 'FA00', 'GO00', 'SY00', 'PW00', 'BA00', 'IF00']
Slate 2: ['SO00', 'RU00', 'SM00', 'VR00', 'ED00', 'BU00', 'AG00', 'II00', 'AS00', 'WM00', 'AP00']

Total pairwise interactions with 2 slates: 164309 

Slate 1: ['HA00', 'RU00', 'SM00', 'ED00', 'BU00', 'GO00', 'PW00', 'BA00']
Slate 2: ['SO00', 'IE00', 'VR00', 'WM00', 'SY00', 'AG00', 'AS00', 'IF00']
Slate 3: ['ZI00', 'IG00', 'HM00', 'JU00', 'FA00', 'II00', 'AP00']

Total pairwise interactions with 3 slates: 145854 



In [10]:
import math

def members( transpose, committees ):
    members = set().union( *[ transpose[ transpose[x] == 1 ].index for x in committees ] )
    return len( members )

def random_equal_partition( dataset, N ):
    dataset = dataset.set_index("MC")
    transpose = dataset.transpose()
    
    committees = list( dataset.index )
    committees.sort( key = lambda x: hash(x + str(N)) )
    
    split = math.ceil( len(committees) / N )
    
    partitions = [ committees[ i * split : (i + 1) * split ] for i in range(N) ]
    partitions = [ (x, members(transpose, x)) for x in partitions ]
    
    return partitions

In [11]:
print("Random Assignment: \n\n")
for N in range(1, 4):
    trial = random_equal_partition( dataset, N )
    for slate in range(N):
        print("Slate {}: {}".format(slate + 1, trial[slate][0]) )
    print( "\nTotal pairwise interactions with {} slates: {} \n".format( N, sum([ x[1] ** 2 for x in trial ]) ) )

Random Assignment: 


Slate 1: ['WM00', 'GO00', 'JU00', 'ED00', 'II00', 'FA00', 'AG00', 'AS00', 'BU00', 'PW00', 'IF00', 'ZI00', 'SY00', 'SO00', 'IG00', 'IE00', 'SM00', 'AP00', 'VR00', 'BA00', 'HA00', 'RU00', 'HM00']

Total pairwise interactions with 1 slates: 187489 

Slate 1: ['HA00', 'SM00', 'JU00', 'IF00', 'VR00', 'SY00', 'IE00', 'BA00', 'GO00', 'PW00', 'IG00', 'II00']
Slate 2: ['ZI00', 'SO00', 'AS00', 'RU00', 'BU00', 'WM00', 'ED00', 'FA00', 'HM00', 'AP00', 'AG00']

Total pairwise interactions with 2 slates: 182536 

Slate 1: ['AG00', 'FA00', 'IF00', 'GO00', 'IG00', 'BA00', 'II00', 'ZI00']
Slate 2: ['SY00', 'HM00', 'RU00', 'AP00', 'BU00', 'PW00', 'ED00', 'VR00']
Slate 3: ['IE00', 'AS00', 'SO00', 'HA00', 'SM00', 'WM00', 'JU00']

Total pairwise interactions with 3 slates: 156554 

